# 모델링을 위한 데이터 선택
데이터셋에 값이 너무 많아 헷갈리고 제대로 출력하기 어려웠습니다. 이 엄청난 양의 데이터를 이해할 수 있는 수준으로 줄일 수 있는 방법은 무엇일까요?

우리의 직관을 이용해 몇 가지 변수를 선택하는 것으로 시작하겠습니다. 이후 코스에서는 변수의 우선순위를 자동으로 지정하는 통계적인 기법을 소개할 것입니다.

변수/열을 선택하기 위해 데이터셋의 모든 열 목록을 확인해야 합니다. 이는 데이터프레임의 **columns** 속성을 확인함으로써 수행할 수 있습니다 (아래 코드).

In [1]:
import pandas as pd

melbourne_file_path = '../../data/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [2]:
# 멜버른 데이터에는 몇 결측값이 존재합니다 (몇 집에 대해 기록되지 않은 값들)
# 결측값을 다루는 방법을 이후 튜토리얼에서 배울 것입니다
# 지금은 가장 간단한 선택지로써 결측값을 포함한 주택 데이터를 버리겠습니다
# 지금은 이에 대해 너무 고민하지 않아도 됩니다

# dropna는 결측값을 버립니다 (na를 "not available"로 생각할 수 있습니다)
melbourne_data = melbourne_data.dropna(axis=0)

데이터의 부분 집합을 선택하는 방법은 여러 가지가 있습니다. [판다스 강의](https://www.kaggle.com/learn/pandas)는 이를 자세히 다루지만, 현재는 두 가지 접근 방식을 중점적으로 살펴보겠습니다.

1. 점 표기법을 사용한 "예측 대상" 선택
2. 열 목록을 사용한 "특성" 선택

## 예측 대상 선택
**점 표기법**을 이용하여 변수를 가져올 수 있습니다. 단일 열은 **시리즈**로 저장되는데, 열이 하나인 데이터프레임과 같습니다.

점 표기법을 이용하여 예측하고자 하는 열을 선택하겠습니다. 관례적으로 예측하려는 값을 **y**라고 합니다. 따라서 멜버른 데이터의 집값을 저장하기 위한 코드는 다음과 같습니다.

In [3]:
y = melbourne_data.Price

# "특성" 선택
(예측에 사용되기 위해) 모델에 입력되는 열을 "특성(features)"이라고 합니다. 우리의 경우, 집값을 예측하기 위해 사용할 열이 됩니다. 때로는 예측 대상을 제외한 모든 열을 사용하기도 합니다. 그렇지 않은 경우 적은 수의 특성만을 사용하는 것이 더 낫습니다.

지금은 몇 가지의 특성만을 이용하는 모델을 제작하겠습니다. 나중에 서로 다른 특성을 이용해 제작한 모델을 비교하는 방법을 알아볼 것입니다.

열 이름 목록을 제공하여 여러 개의 특성을 선택합니다. 목록의 각 항목은 따옴표가 있는 문자열이어야 합니다.

다음은 예시입니다:

In [4]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']

관례적으로 데이터는 **X**라고 합니다.

In [5]:
X = melbourne_data[melbourne_features]

집값을 예측하기 위해 사용할 데이터들을 빠르게 훑어보기 위해 `describe`와 `head` 메소드를 사용합니다.

In [6]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [7]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


이러한 명령어를 사용하여 데이터를 시각적으로 확인하는 것은 데이터 과학자의 업무에서 중요한 부분입니다. 데이터셋에서 추가적인 검사가 필요한 놀라움들을 자주 발견하게 될 것입니다.

---
# 자신만의 모델 제작하기

자신만의 모델을 제작하기 위해 **scikit-learn** 라이브러리를 사용할 것입니다. 샘플 코드에서처럼 이 라이브러리는 **sklearn**으로 적습니다. Scikit-learn은 일반적으로 데이터프레임에 저장된 데이터로 모델링을 할 때 가장 많이 사용되는 라이브러리입니다.

모델을 제작하고 사용하는 순서는 다음과 같습니다:
* **정의**: 어떤 유형의 모델을 만들 것입니까? 의사 결정 트리? 다른 유형의 모델? 모델의 파라미터도 결정합니다.
* **피팅**: 주어진 데이터의 규칙을 찾습니다. 이는 모델링의 꽃입니다.
* **예측**: 말 그대로입니다.
* **평가**: 모델의 예측이 얼마나 정확한지 평가합니다.

다음은 scikit-learn으로 의사 결정 트리 모델을 정의하고 이를 특성 및 목표 변수를 이용해 피팅하는 예시입니다.

In [8]:
from sklearn.tree import DecisionTreeRegressor

# 모델 정의. random_state를 특정하여 늘 같은 결과가 나오도록 합니다
melbourne_model = DecisionTreeRegressor(random_state=1)

# 모델 피팅
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

많은 머신러닝 모델들은 학습 과정에서 무작위성을 허용합니다. `random_state`를 특정하면 언제나 같은 결과를 얻을 수 있음을 보장할 수 있습니다. 이는 좋은 관행으로 여겨집니다. 원하는 숫자를 사용할 수 있으며 모델의 성능은 선택한 값에 유의미한 영향을 받지 않을 것입니다.

우리는 이제 예측에 사용할 수 있는 피팅 된 모델이 있습니다.

실제로, 가격을 이미 알고 있는 주택들보다는 시장에 나올 새로운 주택들의 가격을 예측하고 싶을 것입니다. 그러나 훈련 데이터의 처음 몇 행에 대해 예측 함수가 어떻게 작동하는지 확인하겠습니다.

In [9]:
print("다음 다섯 주택에 대한 예측:")
print(X.head())
print("예측 결과")
print(melbourne_model.predict(X.head()))

다음 다섯 주택에 대한 예측:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
예측 결과
[1035000. 1465000. 1600000. 1876000. 1636000.]
